# PROCESO ETL (Extracción, Transformación y Carga de Datos) - siniestros viales

En este proyecto, nos embarcamos en la tarea crucial de abordar y mitigar los siniestros viales que afectan a la ciudad de Buenos Aires, Argentina. Con el objetivo principal de reducir las tragedias en las vías urbanas, nos sumergiremos en el análisis de datos relacionados con incidentes viales.

Nuestra misión es transformar datos crudos en conocimientos significativos que permitan comprender a fondo los patrones y factores subyacentes que contribuyen a los accidentes de tráfico. A través de un proceso ETL (Extract, Transform, Load), daremos forma a conjuntos de datos dispersos para obtener información valiosa.

El propósito último es proporcionar a las autoridades, organizaciones de tráfico y ciudadanos en general una visión clara y detallada de las áreas de mayor riesgo, los momentos críticos y los elementos que más influyen en la seguridad vial. Con esta información, se pretende impulsar la implementación de medidas preventivas y estrategias eficaces que contribuyan a la reducción significativa de las tragedias viales en la Ciudad Autónoma de Buenos Aires.

### Importamos BIBLIOTECAS necesarias para nuestros primeros pasos

In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functions import cargar_datos_desde_excel
from functions import data_cleaning
from functions import analizar_valores_sd

### CARGA DE LOS DATOS

In [23]:
# Llamamos a la función 'cargar_datos_desde_excel'

datos_homicidios = cargar_datos_desde_excel('homicidios.xlsx', ['HECHOS', 'VICTIMAS'])
datos_lesiones = cargar_datos_desde_excel('lesiones.xlsx', ['HECHOS', 'VICTIMAS'])

### EXPLORACION Y LIMPIEZA DE LOS DATOS

__DataSet:__ HOMICIOS - HECHOS

In [24]:
# Accedemos al dataframe por nombre de hoja, homicidios - hechos

h_hechos_df = datos_homicidios['HECHOS']

In [25]:
#Visualizacion de las primeras filas del dataframe

print("Original DataFrame:")
h_hechos_df.head(3)

Original DataFrame:


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [26]:
#Obtener informacion general del Dataframe

h_hechos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Tras verificar el diccionario de datos, identificamos que los valores "SD" corresponden a "Sin datos". Por consiguiente, tomamos la decisión de cambiar estos valores a NaN para estandarizar.

In [27]:
# Llamamos la funcion analizar valores

resultados_h_hechos = analizar_valores_sd(h_hechos_df)
resultados_h_hechos

,Columna,Cantidad de SD,Porcentaje de SD
6,HORA,1,0.143678
7,HH,1,0.143678
8,LUGAR_DEL_HECHO,1,0.143678
19,VICTIMA,9,1.293103
20,ACUSADO,23,3.304598


In [28]:
#Remplazar SD por NaN en todo el DataFrame

h_hechos_df.replace(['SD','sd'], np.nan, inplace=True)

In [ ]:
# Iniciamos la preparación de los diccionarios y listas que alimentaran la función 'data_cleaning' 

In [29]:
# Columnas para pasar a minúcula

columns_to_lower = ['TIPO_DE_CALLE', 'VICTIMA', 'ACUSADO']

Convertir todos los datos a minúsculas evita posibles problemas de coincidencia y simplificas las operaciones de búsqueda y filtrado.

In [30]:
# Columnas para eliminar

columns_to_drop = ['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO','Calle','Altura',
                   'Cruce','Dirección Normalizada','XY (CABA)','PARTICIPANTES']

Decidimos eliminar las columnas 'AAAA', 'MM', y 'DD' del conjunto de datos, ya que la información de fecha ya está presente en la columna 'FECHA'. Esta decisión simplifica el conjunto de datos, mantiene una estructura más consistente y estandarizada, ahorra espacio y facilita el análisis temporal.

Asimismo, eliminamos la columna 'HORA' debido a la redundancia de la información de hora, que ya está contenida en la columna 'HH'. Esta elección nos proporciona una visión más general y simplificada de la distribución de los siniestros a lo largo del día.

Las columnas 'LUGAR_DEL_HECHO','Calle','Altura','Cruce' y 'Dirección Normalizada' implicaban datos de la dirección del hecho incluso, 'XY (CABA)' que contenía coordenadas en formato de proyección cartesiana también fue eliminada. Preferimos trabajar con las columnas de latitud y longitud por su mayor intuición, no tiene datos nulos, familiaridad para la mayoría de las personas, facilidad de representación gráfica en mapas y conformidad con estándares de sistemas de información geográfica (SIG).

La columna 'PARTICIPANTES' contenia información concatenada redundante de 'VICTIMA' y 'ACUSADO', así que se mantuvieron éstas dos últimas.

In [31]:
# Renombrar columnas

rename_dict = {'ID': 'id_siniestro', 'N_VICTIMAS': 'nro_victimas', 'FECHA': 'fecha', 'HH': 'franja_hora',
               'TIPO_DE_CALLE': 'tipo_calle', 'COMUNA': 'comuna', 'pos x': 'longitud', 'pos y': 'latitud',
               'VICTIMA': 'vehiculo_victima', 'ACUSADO': 'vehiculo_acusado'}

Renombramos columnas para mejorar la claridad y legibilidad del conjunto de datos.

In [32]:
# Convertir a entero

columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']

In [33]:
# Categorización de columnas

columns_to_categorize = ['franja_hora', 'tipo_calle', 'comuna', 'vehiculo_victima', 'vehiculo_acusado']

Al convertir columnas con un conjunto limitado de valores a categorías, se puede facilitar el análisis exploratorio de datos y la comprensión de las distribuciones de esos valores.

In [34]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

h_hechos_dfcleaned = data_cleaning(h_hechos_df,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                   )

c:\Users\Usuario\Desktop\segundo_integrador_henry\Archivos_note\functions.py:206: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = cleaned_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


En el conjunto de datos tenemos dos registros con comuna valor 0. Para este registro '2016-0174' se tenía la calle y, con ésta logramos obtener comuna, longitud y latitud . Para el caso del registro '2016-0151' no tuvimos la misma suerte, por lo cual buscamos el siniestro más cercano '2016-0155' que haya ocurrido también en una calle y le imputamos el mismo valor a las columnas comuna, longitud y latitud. Por la mínima cantidad de registros no vimos necesario hacer uso de alguna api, obtuvimos los datos consultando esta web https://www.comollegara.com/direccion-a-lat-long.html?lugar=autopista%2025%20de%20mayo%20buenos%20aires.

In [35]:
# Reemplazar valores

# Definir los registros que deseas reemplazar
registros_a_reemplazar = [
    {'id_siniestro': '2016-0174', 'comuna': 7, 'longitud': -58.369529920, 'latitud': -34.622472560}, # Comunas 0 
    {'id_siniestro': '2016-0151', 'comuna': 10, 'longitud': -58.52756365, 'latitud': -34.62772274}, # Comunas 0 
    {'id_siniestro': '2018-0039', 'comuna': 14, 'longitud': -58.457579360, 'latitud': -34.538957930}, #sin longitud/latitud
    {'id_siniestro': '2020-0026', 'comuna': 14, 'longitud': -58.457579360, 'latitud': -34.538957930}, #sin longitud/latitud  
    {'id_siniestro': '2017-0042', 'comuna': 14, 'longitud': -58.457579360, 'latitud': -34.538957930}, #sin longitud/latitud  
    {'id_siniestro': '2017-0050', 'comuna': 9, 'longitud': -58.481800300, 'latitud': -34.647005340}, #sin longitud/latitud
    {'id_siniestro': '2017-0051', 'comuna': 7, 'longitud': -58.479447760, 'latitud': -34.678121010}, #sin longitud/latitud      
    {'id_siniestro': '2017-0140', 'comuna': 4, 'longitud': -58.380781350, 'latitud': -34.622364700}, #sin longitud/latitud
    {'id_siniestro': '2020-0039', 'comuna': 9, 'longitud': -58.483096200, 'latitud': -34.671405150}, #sin longitud/latitud   
    {'id_siniestro': '2021-0023', 'comuna': 4, 'longitud': -58.364905600, 'latitud': -34.622974220}, #sin longitud/latitud  
    {'id_siniestro': '2016-0052', 'comuna': 13, 'longitud': -58.457579360, 'latitud': -34.538957930}, #sin longitud/latitud   
    {'id_siniestro': '2016-0136', 'comuna': 4, 'longitud': -58.363401550, 'latitud': -34.624371200}, #sin longitud/latitud  
] 

# Iterar sobre los registros y realizar el reemplazo
for registro in registros_a_reemplazar:
    id_siniestro = registro['id_siniestro']
    comuna = registro['comuna']
    longitud = registro['longitud']
    latitud = registro['latitud']

    # Utilizar loc para reemplazar los valores específicos
    h_hechos_dfcleaned.loc[h_hechos_dfcleaned['id_siniestro'] == id_siniestro, 'comuna'] = comuna
    h_hechos_dfcleaned.loc[h_hechos_dfcleaned['id_siniestro'] == id_siniestro, 'longitud'] = longitud
    h_hechos_dfcleaned.loc[h_hechos_dfcleaned['id_siniestro'] == id_siniestro, 'latitud'] = latitud

In [36]:
print("\nCleaned DataFrame:")
h_hechos_dfcleaned.head()


Cleaned DataFrame:


,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN
4,2016-0005,1,2016-01-21,5,avenida,1,-58.38718297,-34.62246630,moto,pasajeros


__Dataset:__ Homicidios - Víctimas

In [37]:
# Acceder al DataFrame por nombre de hoja, Homicidios - Víctimas

h_victimas_df = datos_homicidios['VICTIMAS']

In [38]:
# Visualiza las primeras filas del Dataframe

print("Original DataFrame:")
h_victimas_df.head(3)

Original DataFrame:


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00


In [39]:
# Obtener información general del DataFrame

h_victimas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


Tras verificar el diccionario de datos, identificamos que los valores "SD" corresponden a "Sin datos". Por consiguiente, tomamos la decisión de cambiar estos valores a NaN para estandarizar.

In [41]:
# Invoca la función 'analizar_valores_sd'

resultados_h_victimas = analizar_valores_sd(h_victimas_df)
resultados_h_victimas

,Columna,Cantidad de SD,Porcentaje de SD
5,ROL,11,1.534170
6,VICTIMA,9,1.255230
7,SEXO,6,0.836820
8,EDAD,53,7.391911
9,FECHA_FALLECIMIENTO,68,9.483961


In [42]:
# Reemplazar 'SD' por NaN en todo el DataFrame

h_victimas_df.replace(['SD','sd'], np.nan, inplace=True)

In [ ]:
# Iniciamos la preparación de los diccionarios y listas que alimentaran la función 'data_cleaning' 

In [43]:
# Columnas para pasar a minúcula

columns_to_lower = ['SEXO','ROL']

Convertir todos los datos a minúsculas evita posibles problemas de coincidencia y simplificas las operaciones de búsqueda y filtrado.

In [44]:
# Columnas para eliminar

columns_to_drop = ['FECHA','AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO', 'VICTIMA']

Las columnas que decidimos eliminar nos parecen irrelevantes para nuestro análisis. Algunas de ellas se complementan con los datos de la hoja 'HECHO'

In [45]:
# Renombrar columnas

rename_dict = {'ID_hecho': 'id_siniestro', 'SEXO': 'sexo',
               'EDAD': 'edad', 'ROL': 'rol'}

Renombramos columnas para mejorar la claridad y legibilidad del conjunto de datos.

In [46]:
# Convertir a entero

columns_to_int = ['edad']

In [47]:
# Columnas para ser agregadas

new_columns_dict = {'gravedad': 'fatal'}

In [48]:
# Llamamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

h_victimas_dfcleaned = data_cleaning(h_victimas_df,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    new_columns=new_columns_dict, # Agregar columnas
                                    )

c:\Users\Usuario\Desktop\segundo_integrador_henry\Archivos_note\functions.py:206: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = cleaned_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [49]:
# Categorización de columnas

columns_to_categorize = ['sexo','edad','gravedad','rol']
h_victimas_dfcleaned = data_cleaning(h_victimas_dfcleaned, categorize_columns=columns_to_categorize) 

c:\Users\Usuario\Desktop\segundo_integrador_henry\Archivos_note\functions.py:206: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_df = cleaned_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Al convertir columnas con un conjunto limitado de valores a categorías, se puede facilitar el análisis exploratorio de datos y la comprensión de las distribuciones de esos valores.

In [50]:
print("\nCleaned DataFrame:")
h_victimas_dfcleaned.head()


Cleaned DataFrame:


,id_siniestro,rol,sexo,edad,gravedad
0,2016-0001,conductor,masculino,19,fatal
1,2016-0002,conductor,masculino,70,fatal
2,2016-0003,conductor,masculino,30,fatal
3,2016-0004,conductor,masculino,18,fatal
4,2016-0005,conductor,masculino,29,fatal


__Dataset:__ Lesiones - Hechos

In [51]:
# Acceder al DataFrame por nombre de hoja, Lesiones - Hechos

l_hechos_df = datos_lesiones['HECHOS']

In [52]:
# Visualiza las primeras filas del Dataframe

print("Original DataFrame:")
l_hechos_df.head(3)

Original DataFrame:


,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD


In [53]:
# Obtener información general del DataFrame

l_hechos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

Tras verificar el diccionario de datos, identificamos que los valores "SD" corresponden a "Sin datos". Por consiguiente, tomamos la decisión de cambiar estos valores a NaN para estandarizar.

In [54]:
# Invoca la función 'analizar_valores_sd'

resultados_l_hechos = analizar_valores_sd(l_hechos_df)
resultados_l_hechos

,Columna,Cantidad de SD,Porcentaje de SD
6,hora,4,0.016817
8,direccion_normalizada,10815,45.469834
9,comuna,846,3.556864
10,tipo_calle,11045,46.436830
11,otra_direccion,18295,76.918226
15,geocodificacion_CABA,1213,5.099853
16,longitud,1209,5.083036
17,latutid,1209,5.083036
18,victima,10733,45.125079
19,acusado,15288,64.275804


In [55]:
# Reemplazar 'SD' por NaN en todo el DataFrame

l_hechos_df.replace(['SD','sd'], np.nan, inplace=True)

In [ ]:
# Iniciamos la preparación de los diccionarios y listas que alimentaran la función 'data_cleaning'